<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/main/Loesung_Uebungen_Teil3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung einlesen der Daten 

In diesem Abschnitt müssen Sie nichts machen. Dieser dient nur für das Einlesen und die Bereitstellung der Daten für die folgenden Übungen. 

In [ ]:
# Bibliotheken einmalig installieren
!pip install pyspark pandas pysqlite3 

In [ ]:
#Angabe der Daten für den Pfad
!wget -O create_data_advacend.py https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Pr%C3%A4sentationen/03-Fortgeschrittene%20Transformationen/create_data_advacend.py

--2022-05-12 11:10:53--  https://github.com/Fuenfgeld/2022TeamADataEngineeringBC/blob/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Pr%C3%A4sentationen/03-Fortgeschrittene%20Transformationen/create_data_advacend.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘create_data_advacend.py’

create_data_advacen     [ <=>                ] 296.55K  --.-KB/s    in 0.06s   

2022-05-12 11:10:53 (5.21 MB/s) - ‘create_data_advacend.py’ saved [303669]



In [ ]:
#Import der nötigen Packete
import sqlite3
import pandas as pd
import pyspark.sql.functions as func
import os

os.system("python3 create_data_advacend.py")

0

In [ ]:
## (Py)Spark starten
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Datei Corn einlesen & in Spark-Dataframe schreiben
df_corn = spark.read.option("multiline",True).json('Corn.json').show()
df_corn.show()

AnalysisException: ignored

In [ ]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

OperationalError: ignored

In [ ]:
#DataFrame für Datenbank erzeugen
df_fields= spark.createDataFrame(df_fields)
df_fields.show()

Nachdem nun alle Daten geladen wurden können Sie anschließend mit den Übungen beginnen.

#Übungen für Fortgeschrittene 

In dieser Übung müssen Sie Jeremy tatkräftig unter die Arme greifen. 
Nachdem wir bereits Ihm bereits bei dem Anbau des Obst und Gemüse geholen haben, müssen Sie nun für in die Daten für das Getreide auswerten.

Hierfür müssen Sie folgend vorgehen:

1.   Helfen Sie Jeremy dabei sich einen Überblick über die Daten zu verschaffen in dem Sie JOIN verwerden.
2.   Berechnen Sie für die jeweilge Getreideart den Wasserverbrauch, Ertrag pro Fläche sowie die Einnahmen.



__________________________________________________________________________




Um Jeremy helfen zu können verschaffen Sie sich zunächst einen Überblick über die Daten.

In [ ]:
df_fields.show(10)
df_corn.show(10)

NameError: ignored

Verknüpfen Sie anschließend die zwei Tabellen miteinander. Wählen Sie hierfür die geeignete Join-Bedingung. 

In [ ]:
gesamt_join = df_corn.join(df_fields, df_corn.field ==  df_fields.field_id,"inner")
gesamt_join.show(30)

Um die Wirtschaftlichkeit unseres Unternehmen zu prüfen, möchte Jeremy sich den gesamten Ertrag pro Fläch für die jeweiligen Getreidesorte wissen. Helfen Sie Jeremy dabei in dem Sie:

+  Die Spalten des Ertrag pro m² mit der Fläche in m² multiplizieren um den gesamten Ertrag pro Fläche zu erhalten 
+ Erzeugen Sie eine neue Spalte im DataFrame für das Ergebnis
+ Tipp: hierfür können Sie die Funcktion col der Sparkfunctions verwenden



In [ ]:
gesamt_join = gesamt_join.withColumn("yield_per_area", func.col("yield_per_sqm") * func.col("area_in_sqm"))
gesamt_join.show()


Nutzen Sie **groupby()** Bedingung sowie die Sparkfuntions **sum** um Jeremy eine Übersicht der Einnahmen, Wasserverbrauchs und dem Ertrag pro Fläche des jeweiligen Getreide zu verschaffen.



In [ ]:
gesamt_ertrag =  gesamt_join.groupBy("crop").agg(func.sum("revenue"), func.sum("yield_per_area"), func.sum("water_consumption"))
gesamt_ertrag = gesamt_ertrag.select(func.col("crop").alias("Crop"),func.col("sum(revenue)").alias("revenue") , func.col("sum(water_consumption)").alias("water_consumption"), func.col("sum(yield_per_area)").alias("yield_per_area"))
gesamt_ertrag.show()

Nachdem wir den Ertrag für die einzelnen Getreidesorten kennen, müssen wir Jeremy bei der Ermittelung des Gewinn je Lebensmittel unter die Arme greifen.

+ Hierfür benötigen wir die zuvor ermittelten Liter je Sorte welche wir mit dem Wert von 0.2 Cent verrechnen. 
 
+ Anschließend müssen noch die Erwerbskosten für die  Pflanzensamen miteinbeziehen um unsere Produktionskosten zu erhalten.
  >Kosten für Samen :
  + Mais = 5 Cent
  + Hafer = 10 Cent
  + Roggen = 20 Cent 


+ Geben Sie das Ergebnis absteigend an. 

Die folgende Codezeile können Sie einfach ausführen um mit den Übungen fortzufahren.

In [ ]:
from pyspark.sql.window import Window as W
rating = [5,10,20]
seed_cost = spark.createDataFrame([(l,) for l in rating], ['cost_of_seed'])
gesamt_ertrag = gesamt_ertrag.withColumn("idx", func.monotonically_increasing_id())
seed_cost = seed_cost.withColumn("idx", func.monotonically_increasing_id())
gesamt_ertrag.show()

windowSpec = W.orderBy("idx")
gesamt_ertrag = gesamt_ertrag.withColumn("idx", func.row_number().over(windowSpec))
seed_cost = seed_cost.withColumn("idx", func.row_number().over(windowSpec))
seed_cost.show()


Nun sind Sie wieder gefragt. Joinen Sie die beiden Tabellen um mit der Ermittlung der Produktionskosten weiterzufahren.

In [ ]:
gesamt_ertrag = gesamt_ertrag.join(seed_cost, gesamt_ertrag.idx == seed_cost.idx).drop("idx")
gesamt_ertrag.show()

In [ ]:
from pyspark.sql.functions import col
gesamt_ertrag = gesamt_ertrag.withColumn("revenue_after_water_cost", func.col("revenue") - (func.col("water_consumption")* 0.02))
gesamt_ertrag = gesamt_ertrag.withColumn("netto_revenue",col("revenue_after_water_cost") - col("cost_of_seed"))
gesamt_ertrag.orderBy(col("netto_revenue").desc()).show()

Um auch für das Getreide in unserem Hofladen gewinnbringend vertreiben können, müssen wir Jeremy bei der Preisfindung unterstützen. 

Die Einnahmen sind bisher in Pence angegeben Rechnenen Sie diese in Pfund um.

>> Randinformationen:
*   Ein Pfund (£) hat 100 Pence (p).
*   Für den Mindestverkaufspreis müssen unsere Getreidesorten noch mit einer Mehrwertsteuer von 19% besteuert werden.
* Für einen lukrativen Verkaufspreis müssen wir den Nettopreis mit 59% besteuern






 





+ Verwenden Sie für die Preisfindung Lambda-Funktionen. Welche der beiden Variante Sie verwenden bleibt Ihnen überlassen. 

+ Ermitteln Sie hierfür den Preis pro Stück und erzeugen hierfür eine neue Spalte in dem Dataframe

In [ ]:
rdd2 = gesamt_ertrag.rdd.map(lambda x: (x["Crop"], x["netto_revenue"]*1.19*0.01, x["netto_revenue"]*1.59*0.01)) 
cost_overview = rdd2.toDF(["Crop","min_selling_price_per_piece", "lucrative_selling_price"])
cost_overview = cost_overview.withColumn("min_selling_price_per_piece", func.round(cost_overview["min_selling_price_per_piece"], 2)).withColumn("lucrative_selling_price", func.round(cost_overview["lucrative_selling_price"], 2))
cost_overview.withColumn("currency", func.lit("£")).orderBy(col("lucrative_selling_price").desc()).show()

In [ ]:
# By Calling function
def calc_func(x):
    Crop = x.Crop
    Netto_Revenue = x.netto_revenue 
    min_selling_price_per_piece = Netto_Revenue*1.19*0.01
    lucrative_selling_price = Netto_Revenue*1.59*0.01
    return (Crop, min_selling_price_per_piece, lucrative_selling_price)

rdd_process=gesamt_ertrag.rdd.map(lambda x: calc_func(x))
cost_overview_with_func = rdd_process.toDF(["Crop","min_selling_price_per_piece", "lucrative_selling_price"])
cost_overview_with_func = cost_overview_with_func.withColumn("min_selling_price_per_piece", func.round(cost_overview_with_func["min_selling_price_per_piece"], 2)).withColumn("lucrative_selling_price", func.round(cost_overview_with_func["lucrative_selling_price"], 2))
cost_overview_with_func.withColumn("currency", func.lit("£")).orderBy(col("lucrative_selling_price").desc()).show()
